In [210]:
import os
import hmac
import hashlib
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

In [211]:
class IoTDevice:
    def __init__(self, device_id, shared_vault):
        self.device_id = device_id
        self.vault = shared_vault.copy()
        self.C2 = None
        self.r2 = None
        self.t1 = None
        self.session_key = None
        self.trasmitted_data = None
        

    def create_connection_request(self, session_id):
        print(f"IoTDevice: Creating connection request with session_id: {session_id} and device_id: {self.device_id}")
        return {"device_id": self.device_id, "session_id": session_id}

    def generate_response(self, challenge_indices, r1):
        print(f"IoTDevice: Generating response with challenge_indices: {challenge_indices} and r1: {r1}")
        # Step 1: Compute k1 as XOR of keys in the indices
        k1 = self.vault[challenge_indices[0]]
        for i in challenge_indices[1:]:
            k1 = bytes(a ^ b for a, b in zip(k1, self.vault[i]))
        print("IoTDevice: K1 (XOR Vault): ", k1)
        print("IoTDevice: Length of K1: ", len(k1))
        if len(k1) not in (16, 24, 32):
            raise ValueError("IoTDevice: K1 must be 16, 24, or 32 bytes long for AES encryption")
        # Step 2: Generate t1 (random number for session key generation)
        t1 = os.urandom(len(r1))
        self.t1 = t1
        print("IoTDevice: R1 from server: ", r1)
        print("IoTDevice: T1: ", t1)
        print("IoTDevice: Length of T1: ", len(t1))

        # Step 3: Concatenate r1 and t1
        concat_r1_t1 = r1 + t1
        
        print("IoTDevice: Concatenated R1 and T1: ", concat_r1_t1)

        # Step 5: Generate C2
        c2, r2 = self.generate_challenge()
        self.C2 = c2
        self.r2 = r2
        print("IoTDevice: Generated C2: ", c2)
        print(f"IoTDevice: Generated R2: {r2}")
        # Step 6: Concatenate response and c2
        
        data_to_encrypt = concat_r1_t1 + bytes(c2) + bytes(r2)
        print("IoTDevice: Data to Encrypt: ", data_to_encrypt)
        cipher = AES.new(k1, AES.MODE_CBC)  # Use CBC mode for encryption
        iv = cipher.iv  # Initialization vector for CBC
        encrypted_data = cipher.encrypt(pad(data_to_encrypt, AES.block_size))
        print("IoTDevice: Encrypted Data: ", encrypted_data)

        # Include the IV with the encrypted data for decryption
        final_response = iv + encrypted_data
        print("IoTDevice: Final Response (IV + Encrypted Data): ", final_response)
        return {"response": final_response}
    
    def generate_challenge(self, p=3):
        indices = list(set(os.urandom(1)[0] % len(self.vault) for _ in range(p)))
        while len(indices) < p:
            indices.append(os.urandom(1)[0] % len(self.vault))  # Ensure distinct indices
        r1 = os.urandom(8)  # Random number
        print(f"IoTDevice: Client selected indices indices: {indices}")
        print(f"IoTDevice: Random number r1: {r1}")
        return indices, r1
    
    def verify_response(self, response):
        k2 = self.vault[self.C2[0]]
        for i in self.C2[1:]:
            k2 = bytes(a ^ b for a, b in zip(k2, self.vault[i]))
        print("IoTDevice: K2 Partial (XOR Vault): ", k2)
        # Step 1: k2 XOR t1
        k2 = bytes(a ^ b for a, b in zip(k2, self.t1))
        print(f"IoTDevice: K2 (XOR Vault XOR T1): {k2}, Length: {len(k2)}")
        if len(k2) < 16:
            k2 = k2.ljust(16, b'\0')  # Pad with zeroes
        elif len(k2) > 32:
            k2 = k2[:32]  # Truncate to 32 bytes
        elif len(k2) not in (16, 24, 32):
            k2 = k2.ljust(24, b'\0')  # Default to 24 bytes
        print(f"IoTDevice: K2 (XOR Vault XOR T1) after padding: {k2}, Length: {len(k2)}")
        # Step 1: Decrypt the response
        iv = response[:AES.block_size]
        cipher = AES.new(k2, AES.MODE_CBC, iv=iv)
        decrypted_data = unpad(cipher.decrypt(response[AES.block_size:]), AES.block_size)
        print("IoTDevice: Decrypted Data: ", decrypted_data)
        r2 = decrypted_data[:8]
        print("IoTDevice: R2 from server: ", r2)
        if r2 != bytes(self.r2):
            raise ValueError("IoTDevice: R2 does not match")
        print("IoTDevice: R2 matches")
        
        # Step 3: Session key t1 XOR t2
        t2 = decrypted_data[8:]
        print("IoTDevice: T2 from server: ", t2)
        print("IoTDevice: T1: ", self.t1)
        session_key = bytes(a ^ b for a, b in zip(self.t1, t2))
        if (len(session_key) < 16):
            session_key = session_key.ljust(16, b'\0')
        print("IoTDevice: Session Key: ", session_key)
        self.session_key = session_key
        
    def encrtpt_msg(self, msg):
        self.trasmitted_data = msg
        cipher = AES.new(self.session_key, AES.MODE_CBC)
        iv = cipher.iv
        encrypted_data = cipher.encrypt(pad(msg, AES.block_size))
        return iv + encrypted_data
    
    def decrypt_msg(self, msg):
        iv = msg[:AES.block_size]
        cipher = AES.new(self.session_key, AES.MODE_CBC, iv=iv)
        decrypted_data = unpad(cipher.decrypt(msg[AES.block_size:]), AES.block_size)
        self.trasmitted_data = decrypted_data
        return decrypted_data

    def update_vault(self):
        # Digest all the secure vault data
        data_to_digest = b''.join(self.vault)
        h_key = hmac.new(self.trasmitted_data, data_to_digest, hashlib.sha256).digest()
        print(f"IoTDevice: Generated HMAC key: {h_key}")
        j = 10
        k = 16
        data_parts = [data_to_digest[i:i+k] for i in range(0, len(data_to_digest), k)]
        print(f"IoTClient: Data Parts: {data_parts}")
        # For each part XOR with the HMAC key and update the vault
        for i, part in enumerate(data_parts):
            self.vault[i] = bytes(a ^ b for a, b in zip(part, h_key))
        print(f"IoTClient: Updated Vault: {self.vault}")
        

In [212]:
class IoTServer:
    def __init__(self, shared_vault, valid_device_ids):
        self.vault = shared_vault.copy()
        self.valid_device_ids = valid_device_ids
        self.C1 = None
        self.C2 = None
        self.r1 = None
        self.r2 = None
        self.session_key = None
        self.trasmitted_data = None

    def handle_connection_request(self, request):
        device_id = request.get("device_id")
        session_id = request.get("session_id")
        if device_id in self.valid_device_ids:
            print(f"IoTServer: Connection request accepted for Device ID: {device_id}, Session ID: {session_id}")
            return True
        else:
            print(f"IoTServer: Connection request denied for Device ID: {device_id}")
            return False

    def generate_challenge(self, p=3):
        indices = list(set(os.urandom(1)[0] % len(self.vault) for _ in range(p)))
        while len(indices) < p:
            indices.append(os.urandom(1)[0] % len(self.vault))  # Ensure distinct indices
        r1 = os.urandom(8)  # Random number
        print(f"IoTServer: Server selected indices indices: {indices}")
        print(f"IoTServer: Random number r1: {r1}")
        return indices, r1

    def verify_response(self, response):
        # Step 1: Compute k1 as XOR of keys in the indices
        k1 = self.vault[self.C1[0]]
        for i in self.C1[1:]:
            k1 = bytes(a ^ b for a, b in zip(k1, self.vault[i]))
        print("IoTServer: K1 Generated from Local Challenge (XOR Vault): ", k1)
        # Retireve r1 from response
        # Assuming shared_key and final_response are known
        iv = response[:16]  # Extract IV (16 bytes for AES)
        encrypted_data = response[16:]  # Remaining is the encrypted data
        cipher = AES.new(k1, AES.MODE_CBC, iv)
        decrypted_data = unpad(cipher.decrypt(encrypted_data), AES.block_size)
        print("IoTServer: Decrypted Data: ", decrypted_data)
        #Extract r1 and t1 from decrypted data
        r1 = decrypted_data[:8]
        t1 = decrypted_data[8:16]
        c2 = decrypted_data[16:19]
        r2 = decrypted_data[19:]
        print("IoTServer: Extracted R1: ", r1)
        print("IoTServer: Extracted T1: ", t1)
        print("IoTServer: Extracted C2: ", c2)
        print("IoTServer: Extracted R2: ", r2)
        if r1 != self.r1:
            print("IoTServer: R1 does not match")
            return False
        else:
            print("IoTServer: R1 matches")
            
        # Step 2: Compute k2 as XOR of keys in the indices
        c2 = list(c2)
        print("IoTServer: C2: ", c2)
        k2 = self.vault[c2[0]]
        for i in c2[1:]:
            k2 = bytes(a ^ b for a, b in zip(k2, self.vault[i]))
        print("IoTServer: K2 (XOR Vault for C2): ", k2)
        # Step 3: Generate t2 (random number for session key generation)
        t2 = os.urandom(len(t1))
        print("IoTServer: T2: ", t2)
        # Step 4: XOR k2 with t1 to generate key
        k2 = bytes(a ^ b for a, b in zip(k2, t1))
        # Adjust K2 length to be valid for AES
        if len(k2) < 16:
            k2 = k2.ljust(16, b'\0')  # Pad with zeroes
        elif len(k2) > 32:
            k2 = k2[:32]  # Truncate to 32 bytes
        elif len(k2) not in (16, 24, 32):
            k2 = k2.ljust(24, b'\0')  # Default to 24 bytes
        data_to_encrypt = r2 + t2
        print("IoTServer: Data to Encrypt: ", data_to_encrypt)
        cipher = AES.new(k2, AES.MODE_CBC)
        iv = cipher.iv
        encrypted_data = cipher.encrypt(pad(data_to_encrypt, AES.block_size))
        print("IoTServer: Encrypted Data: ", encrypted_data)
        print(f"IoTServer t1 for session key: {t1}")
        print(f"IoTServer t2 for session key: {t2}")
        session_key = bytes(a ^ b for a, b in zip(t1, t2))
        if (len(session_key) < 16):
            session_key = session_key.ljust(16, b'\0')
        self.session_key = session_key
        print("IoTServer: Session Key: ", session_key)
        
        return iv + encrypted_data
    
    def encrtpt_msg(self, msg):
        self.trasmitted_data = msg
        cipher = AES.new(self.session_key, AES.MODE_CBC)
        iv = cipher.iv
        encrypted_data = cipher.encrypt(pad(msg, AES.block_size))
        return iv + encrypted_data
    
    def decrypt_msg(self, msg):
        iv = msg[:AES.block_size]
        cipher = AES.new(self.session_key, AES.MODE_CBC, iv=iv)
        decrypted_data = unpad(cipher.decrypt(msg[AES.block_size:]), AES.block_size)
        self.trasmitted_data = decrypted_data
        return decrypted_data

    def update_vault(self):
        # Digest all the secure vault data
        data_to_digest = b''.join(self.vault)
        h_key = hmac.new(self.trasmitted_data, data_to_digest, hashlib.sha256).digest()
        print(f"IoTServer: New HMAC Key: {h_key}")
        # Split the data to digest in j equal parts of k bytes
        j = 10
        k = 16
        data_parts = [data_to_digest[i:i+k] for i in range(0, len(data_to_digest), k)]
        print(f"IoTServer: Data Parts: {data_parts}")
        # For each part XOR with the HMAC key and update the vault
        for i, part in enumerate(data_parts):
            self.vault[i] = bytes(a ^ b for a, b in zip(part, h_key))
        print(f"IoTServer: Updated Vault: {self.vault}")
        

In [213]:
# Shared initialization
VAULT_SIZE = 10
KEY_SIZE = 16
shared_vault = [os.urandom(KEY_SIZE) for _ in range(VAULT_SIZE)]
print("Shared Vault: ")
for i, key in enumerate(shared_vault):
    print(f"Index {i}: {key}")
valid_device_ids = {"Device_1", "Device_2"}  # List of valid device IDs

# Create device and server with the shared vault
device = IoTDevice(device_id="Device_1", shared_vault=shared_vault)
server = IoTServer(shared_vault=shared_vault, valid_device_ids=valid_device_ids)

Shared Vault: 
Index 0: b'\xde\x03z\xca\xc8k\xab\xe6I\xdd\xe7\x0c\xd4\xed\x16>'
Index 1: b'\xadl\x93\x07Slk\xad\xec\xa3d\x8d6\xb8\xba3'
Index 2: b'\xc8\x9a\xfbiK\xfc\x8c9 ^u\xf2\xe7\x0b\xe3\xc4'
Index 3: b'\xe6\x9c\x02\xe9b4\xe2vL\x01\x90\x16\x99\xb5x\xb1'
Index 4: b'`\xaa\x98\x16V3\x93\xfe\xe7\xafp9m\xbc\xbcf'
Index 5: b"\xd6-\xccw\x0b'T\xe3\x16G]c\xa6\xfeb\x14"
Index 6: b'\xdc;\x8c\x91\xb5X[,\xc3_\xd5\x12\x7f\xba\xb7\x10'
Index 7: b'V\xedm!\x82J\x16\x06\xa4\x9dR&qeUm'
Index 8: b'`\x1b?\xa22ko3I\xd5A\x12\xfewz\xce'
Index 9: b'\xf8h\xdc\xcb\xa2\xbdk\xe6B\xcf&\xe3\x07\xee \xbc'


In [214]:
# Authentication process
print("Starting Authentication Process")

# Step 1: Device sends connection request
session_id = "session_123"
connection_request = device.create_connection_request(session_id)

# Step 2: Server verifies device ID and sends challenge if valid
if server.handle_connection_request(connection_request):
    challenge_indices, r1 = server.generate_challenge()
    server.r1 = r1
    server.C1 = challenge_indices

    # Step 3: Device generates response
    response_packet = device.generate_response(challenge_indices, r1)
    device_response = response_packet["response"]
    print("[AUTH] Device Response: ", device_response)
    
    
    # Step 4: Server verifies response
    ser_Res = server.verify_response(device_response)
    device.verify_response(ser_Res)
    msg = b"Hello from IoT Device"
    print(f"[Communication] Sending msg from Device to Server: {msg}")
    enc_msg = device.encrtpt_msg(msg)
    server_dec_msg = server.decrypt_msg(enc_msg)
    print(f"[Communication] Server Decrypted Message: {server_dec_msg}")
    # Updating vault
    device.update_vault()
    server.update_vault()
else:
    print("IoTServer: Connection denied.")

Starting Authentication Process
IoTDevice: Creating connection request with session_id: session_123 and device_id: Device_1
IoTServer: Connection request accepted for Device ID: Device_1, Session ID: session_123
IoTServer: Server selected indices indices: [1, 3, 6]
IoTServer: Random number r1: b'\xa7\xe7\x8aV\xcd`\xad\xab'
IoTDevice: Generating response with challenge_indices: [1, 3, 6] and r1: b'\xa7\xe7\x8aV\xcd`\xad\xab'
IoTDevice: K1 (XOR Vault):  b'\x97\xcb\x1d\x7f\x84\x00\xd2\xf7c\xfd!\x89\xd0\xb7u\x92'
IoTDevice: Length of K1:  16
IoTDevice: R1 from server:  b'\xa7\xe7\x8aV\xcd`\xad\xab'
IoTDevice: T1:  b'\xb8\xddC\xc0%\xef!\x02'
IoTDevice: Length of T1:  8
IoTDevice: Concatenated R1 and T1:  b'\xa7\xe7\x8aV\xcd`\xad\xab\xb8\xddC\xc0%\xef!\x02'
IoTDevice: Client selected indices indices: [1, 4, 5]
IoTDevice: Random number r1: b'H\x1e\xa6\xe4E_\xc1\x16'
IoTDevice: Generated C2:  [1, 4, 5]
IoTDevice: Generated R2: b'H\x1e\xa6\xe4E_\xc1\x16'
IoTDevice: Data to Encrypt:  b'\xa7\xe7\